## 第一階段：取得所有必要的職缺網址

In [167]:
from datetime import datetime
import json
import random
import re
import time
from urllib.parse import unquote, quote

from bs4 import BeautifulSoup
import requests

In [140]:
search_url = input('請輸入搜尋頁面網址 ')

請輸入搜尋頁面網址 https://www.104.com.tw/jobs/search/?ro=0&keyword=%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB&area=6001002000%2C6001001000&jobsource=2018indexpoc


In [185]:
kw = input('請輸入關鍵字 ')
search_url = \
f'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={quote(kw)}&area=\
6001002000%2C6001001000&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc'

請輸入關鍵字 Python


In [3]:
def url_to_soup(url):
    head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} 
    resp = requests.get(url, headers=head)
    soup = BeautifulSoup(resp.text)
    return soup

In [4]:
def get_page_urls(search_url):
    soup = url_to_soup(search_url)
    jobs = soup.select('.js-job-link')
    page_urls = []
    for job in jobs:
        try:
            page_urls.append('https:' + job['href'].split('&')[0])
        except:
            page_urls.append('https:' + job['href'])
    return page_urls

In [186]:
def get_all_urls(search_url):
    job_urls = []
    for page in range(1, 151):
        if 'page=' in search_url:
#           用 f-string 就不必使用 str(page)了，因為page必轉為str
            search_url = re.sub(r'page=\d*', f'page={page}',
                                search_url)
        else:
            search_url = search_url + f'&page={page}'
            
        print(f'{page}頁', end='\t')
        page_urls = get_page_urls(search_url)
        job_urls.extend(page_urls)
        
        if len(page_urls) < 20:
            break
        time.sleep(random.uniform(4, 10))

    print('')
    print('總共收集', len(job_urls), '個職缺網址')
    print('完成')
    return job_urls

In [ ]:
result = get_all_urls(search_url)

1頁	2頁	3頁	4頁	5頁	6頁	7頁	8頁	9頁	10頁	11頁	12頁	13頁	14頁	15頁	16頁	17頁	18頁	19頁	20頁	21頁	22頁	23頁	24頁	25頁	26頁	27頁	28頁	29頁	30頁	31頁	32頁	33頁	34頁	

### 將結果寫入JSON檔

#### 先從網址取出關鍵字

In [168]:
keyword_raw = re.search(r'keyword=[^&]*', search_url).group().split('=')[1]
keyword = unquote(keyword_raw)
keyword

'Python'

#### 記錄當下時間，以指定格式

In [169]:
timestamp = datetime.now().strftime('%m%d-%H%M')
timestamp

'0428-0203'

In [170]:
len(result)

2283

In [171]:
result_set = set(result)

In [172]:
len(result_set)

2016

In [173]:
result_set.discard('https:javascript:void(0);')
result_tuple = tuple(result_set)
len(result_tuple)

2015

In [174]:
with open(f'job_urls/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(result_tuple, f)

In [175]:
# filename = input()
# with open(f'data/{filename}.json', encoding='utf8') as file:
#     one = json.load(file)

In [176]:
block = ('前端', '測試', '分析', 'front', 'php', 'java', 'asp', 'c#', 'ios', 
        'android', 'react', 'vue', 'angular', 'ux', 'ui', 'rail',
        'ruby', '.net', 'core', 'test', '視覺', '資安', '網頁設計', 'html',
        'css', 'jquery', '影像', '行政', 'manager', '韌體', 'c+', 'unity',
        '硬體', '介面', 'qa', 'quality', '維護', '研發替代役', 'designer',
        '美術', '物聯網', 'iot', '介面設計', '嵌入', 'devops', 'scientist',
        '人工智', '圖像', '算法', 'hardware', '硬體', '品質', 'lte', '實習',
        '科學家', 'firmware', 'sre', 'reliability', '可靠性', 'embed', '顧問',
        'vr', '助理', '自動駕', '深度學習', 'nlp', '工讀', 'intern', '兼職',
        '襄理', '副理', '經理', 'fae', 'bot', 'crm', '主管', '主任', 'analyst',
        '網管', 'integr', 'pm', '3d', '動畫', '行銷', '研究員', '機率', '特效',
        'mis', '光學', '約聘', '總監', 'algorithm','檢測', '支援', '客服', '製作人',
        'erp', '網站設計', '編輯', '專員', '總監', '小編', '機器人', '企劃', 'node',
        '補習班', '證券', '老師', '自動化', '機器學習', '講師', 'learning', '教師',
        '交易員', '驗證', '汽車', '財務工程')

# app 遊戲 bi

In [177]:
filename = input('請輸入檔名：')
with open(f'job_urls/{filename}.json', encoding='utf8') as file:
    urls = json.load(file)
len(urls)

請輸入檔名：後端工程師-0425-2321


1503

In [178]:
def urls_to_data(result, block, block_urls):
    print(datetime.now().strftime('%H:%M:%S'))
    print(f'開始處理，總共有 {len(result)} 個項目')
    block_list = block
    data = []
    item = 1
    number = 1
    for url in result:
        blocked = False
        job = {}
        
        if url in block_urls:
            print('◆', end='\t')
            number = number + 1
            continue
        
        try:
            soup = url_to_soup(url)
            job['資料編號'] = f'第 {item} 項'
            job['職位網址'] = f' {url} '            
            job['職位名稱'] = soup.find_all('meta')[4]['content'].rstrip(
                ' - 104人力銀行')
            
            if 'python' in job['職位名稱'].lower():
                pass
            else:
#                 blocked = any(( item for item in block_list 
#                                if item in job['職位名稱'].lower()))
                for term in block_list:
                    if term in job['職位名稱'].lower():
                        blocked = True
                        break
            if blocked:
                print('×', end='\t')
                number = number + 1
                time.sleep(random.uniform(3, 9))
                continue

            job['工作地點'] = soup.find('dd', 'addr').text.strip().split()[0]
            job['工作內容'] = soup.find('p').text.replace('\r', '').replace(
                '\t', '').replace('\u3000', '').replace('\xa0', '')
            job['工作內容'] = job['工作內容'][:300]
            
            job['擅長工具'] = soup.select('.info')[1].find_all('dd')[-3].text
            job['工作技能'] = soup.select('.info')[1].find_all('dd')[-2].text
            job['其它條件'] = soup.select('.info')[1].find_all(
                'dd')[-1].text.replace('\r', '').replace('\t', '').replace(
                '\u3000', '').replace('\xa0', '')
            job['其它條件'] = job['其它條件'][:300]
            
        except:
            print('×', end='\t')
            number = number + 1
            time.sleep(random.uniform(3, 9))
            continue

        for k, v in job.items():
            if v in ('不拘', '未填寫'):
                job[k] = ''
        data.append(job)
        print(number, end='\t')
        item = item + 1
        number = number + 1
        time.sleep(random.uniform(3, 9))
    print('')
    print(f'完成，實際抓取 {len(data)} 個項目')
    print(datetime.now().strftime('%H:%M:%S'))
    return data

In [179]:
# data = urls_to_data(one, block=block)

In [180]:
data = urls_to_data(result_tuple, block=block, block_urls=urls)

02:04:43
開始處理，總共有 2015 個項目
×	×	3	4	5	×	×	8	9	◆	11	◆	×	×	×	×	×	18	×	×	×	22	×	×	×	26	×	28	×	×	×	◆	◆	×	×	×	×	×	39	◆	◆	×	×	×	×	×	×	48	×	×	51	×	×	◆	×	×	◆	58	×	60	

KeyboardInterrupt: 

In [18]:
data

[{'資料編號': '第 1 項',
  '職位網址': ' https://www.104.com.tw/job/?jobno=5mgyh ',
  '職位名稱': '軟體工程師-學生實習 | 恩智區塊鏈科技有限公司 | 新北市新店區',
  '工作地點': '新北市新店區北新路三段213號10樓',
  '工作內容': '1.技術研發實習生：APP/Web/後端/區塊鏈開發與測試。2.資工/資管相關科系尤佳，無經驗可。 3.工作態度積極、負責，願意主動學習並重視團隊合作。 4.一週可上班時數至少20小時以上。',
  '擅長工具': '',
  '工作技能': '',
  '其它條件': ''},
 {'資料編號': '第 2 項',
  '職位網址': ' https://www.104.com.tw/job/?jobno=6kew8 ',
  '職位名稱': '智慧城市專案管理師 | 思納捷科技股份有限公司 | 台北市松山區',
  '工作地點': '台北市松山區民生東路4段133號4樓',
  '工作內容': '1. 負責專案需求管理2. 負責專案建置時程規劃管理(場勘、備料、派工、整合、設定、測試)3. 負責專案執行時與客戶溝通協調與需求分析4. 負責專案執行中技術問題解決或反應5. 負責專案現場監工，必要時支援施作6. 負責專案相關人力調配與內部溝通7. 負責專案之結案管理',
  '擅長工具': '',
  '工作技能': '普通重型機車、普通小型車',
  '其它條件': '1. 熟悉MicroSoft office Word,PowerPoint,Excel操作2. 熟悉AutoCAD製圖尤佳其他條件1. 具備專案管理及現場經驗。2. 具備電學與資訊相關知識。3. 熟悉一般乙太網路設定，了解有線網路(Ethernet、RS485)及無線傳輸技術(Wifi、ZigBee、LoRa、Bluetooth)概念及應用方法。4. 略懂軟體系統運作原理包含前後端架構與資料庫系統。5. 熟悉標案流程具提案經驗者佳。6. 了解配電盤、馬達、冰水主機、空壓機、熱泵或不斷電系統UPS專業知識尤佳。7. 具備主動積極態度，具責任心、細心。 8. 具備團隊合作精神，且可獨立作業。'},
 {'資料編號': '第 3 項',
  '職位網址': ' https://www

In [138]:
timestamp=datetime.now().strftime('%m%d-%H%M')

with open(f'data/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(data, f)

In [36]:
from collections import Counter

In [87]:
Counter(i['工作地點'][:6] for i in data).most_common()

[('台北市內湖區', 170),
 ('台北市信義區', 111),
 ('台北市大安區', 97),
 ('台北市中山區', 75),
 ('台北市南港區', 65),
 ('台北市松山區', 57),
 ('新北市中和區', 44),
 ('台北市中正區', 43),
 ('新北市新店區', 42),
 ('新北市汐止區', 36),
 ('台北市北投區', 19),
 ('新北市土城區', 19),
 ('新北市板橋區', 19),
 ('新北市三重區', 11),
 ('台北市士林區', 11),
 ('新北市五股區', 9),
 ('台北市大同區', 6),
 ('新北市深坑區', 6),
 ('新北市新莊區', 5),
 ('新北市樹林區', 4),
 ('新北市林口區', 4),
 ('台北市萬華區', 4),
 ('新北市永和區', 3),
 ('台北市文山區', 3),
 ('新北市泰山區', 2),
 ('新北市蘆洲區', 1),
 ('新北市淡水區', 1),
 ('新北市八里區', 1),
 ('新北市三峽區', 1),
 ('新北市瑞芳區', 1)]

In [21]:
import json

In [24]:
json.dumps(aa)

'[1, 2, 3]'

In [25]:
with open('data.json', 'w') as f:
    json.dump(aa, f)